Setup method (grabs 9-letter words)

In [1]:
import urllib.request
def get_words():
    urllib.request.urlretrieve("https://github.com/dwyl/english-words/raw/master/words_alpha.txt", "words_alpha.txt")
    with open('words_alpha.txt', 'r') as istream:
        for line in map(str.strip, istream):
            if len(line) == 9:
                yield line

In [2]:
from vis_utils import *
from itertools import islice

ppd(set(islice(get_words(), 13)))

[
  "abaisance",
  "aaronical",
  "abandoner",
  "abacinate",
  "abaciscus",
  "aardvarks",
  "abandonee",
  "aasvogels",
  "abactinal",
  "abalation",
  "abandoned",
  "abamperes",
  "aaronitic"
]


class / method interface

In [3]:
from dataclasses import dataclass

class Solver:
    def __init__(self, words):
        pass

    def solve(self, puzzle: str) -> str:
        pass

In [4]:
Solver(get_words()).solve('IELGAPPAN')
# -> APPEALING

In [5]:
from itertools import permutations

class MySolver(Solver):
    def __init__(self, words):
        self.words = set(words)
        
    def solve(self, puzzle: str):
        seen = []
        for possible in permutations(puzzle.lower()):
            if ''.join(possible) in self.words:
                result = ''.join(possible)
                if result not in seen:
                    yield result
                    seen.append(result)

In [6]:
list(MySolver(get_words()).solve('IELGAPPAN'))

['lagniappe', 'appealing', 'panplegia']

In [7]:
%load_ext memory_profiler

%timeit list(MySolver(get_words()).solve('IELGAPPAN'))
%memit list(MySolver(get_words()).solve('IELGAPPAN'))

solver = MySolver(get_words())
%timeit list(solver.solve('IELGAPPAN'))
%memit list(solver.solve('IELGAPPAN'))

1.86 s ± 153 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 137.03 MiB, increment: 0.71 MiB
540 ms ± 2.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 137.44 MiB, increment: 0.00 MiB


Implement you own solution! aim for low time, low memory, or both!

## My Solution

I'm going to need my own permutation function so that I can hook into the behaviour

In [8]:
def perm(a):
    def sub(i):
        if i == len(a) - 1:
            yield tuple(a)
        else:
            for k in range(i, len(a)):
                a[i], a[k] = a[k], a[i]
                yield from sub(i + 1)
                a[i], a[k] = a[k], a[i]
    yield from sub(0)

In [9]:
list(perm(list('lsk')))

[('l', 's', 'k'),
 ('l', 'k', 's'),
 ('s', 'l', 'k'),
 ('s', 'k', 'l'),
 ('k', 's', 'l'),
 ('k', 'l', 's')]

In [10]:
'''
Let's make a version to hook into that can skip a permutation branch if the first 2 chars don't seem good
'''


def perm(a, mapping):
    def sub(i):
        if i == 2:
            print(''.join(a[:2]), mapping)
            if ''.join(a[:2]) not in mapping:
                return
        if i == len(a) - 1:
            yield tuple(a)
        else:
            for k in range(i, len(a)):
                a[i], a[k] = a[k], a[i]
                yield from sub(i + 1)
                a[i], a[k] = a[k], a[i]
    yield from sub(0)

In [11]:
list(perm(list('abcd'), ['ac']))

ab ['ac']
ac ['ac']
ad ['ac']
ba ['ac']
bc ['ac']
bd ['ac']
cb ['ac']
ca ['ac']
cd ['ac']
db ['ac']
dc ['ac']
da ['ac']


[('a', 'c', 'b', 'd'), ('a', 'c', 'd', 'b')]

In [12]:
from itertools import permutations

class MySolver2(Solver):
    def __init__(self, words):
        self.words, self.mapping = set(words), set()
        for w in self.words:
            if len(w) == 9:
                self.mapping.add(w[:2])
        
    def solve(self, puzzle: str):
        seen = []
        for possible in self.perm(list(puzzle.lower())):
            if ''.join(possible) in self.words:
                result = ''.join(possible)
                if result not in seen:
                    yield result
                    seen.append(result)

    def perm(self, a):
        def sub(i):
            if i == 2:
                # print(''.join(a[:2]), mapping)
                if ''.join(a[:2]) not in self.mapping:
                    return
            if i == len(a) - 1:
                yield tuple(a)
            else:
                for k in range(i, len(a)):
                    a[i], a[k] = a[k], a[i]
                    yield from sub(i + 1)
                    a[i], a[k] = a[k], a[i]
        yield from sub(0)

In [13]:
list(MySolver2(get_words()).solve('IELGAPPAN'))

['lagniappe', 'appealing', 'panplegia']

In [15]:
s = MySolver(get_words())
s2 = MySolver2(get_words())

In [16]:
%timeit list(s.solve('KDJSNDILK'))
%timeit list(s2.solve('KDJSNDILK'))

542 ms ± 4.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.18 s ± 2.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
